# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

In [186]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [6]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/01 Practice/00_data/"
DATA_PATH

'C:/Users/user/Desktop/데이터분석/01 Practice/00_data/'

- 데이터 불러오기

In [7]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [8]:
train_tr.isnull().sum().sum(), test_tr.isnull().sum().sum(), 

(0, 0)

# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [9]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [14]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

## 구매일시를 이용한 특성생성

In [192]:
# 2004년 5월 ~ 2005년 4월 공휴일 리스트
holiday_2004 = ["2004-05-05", "2004-05-26", "2004-06-06",
                "2004-07-17", "2004-08-15", "2004-09-27",
                "2004-09-28", "2004-09-29", "2004-10-03", "2004-12-25"]

holiday_2005 = ["2005-01-01", "2005-02-08", "2005-02-09",
                "2005-02-10", "2005-03-01", "2005-04-05"]

holidays = pd.to_datetime(holiday_2004 + holiday_2005)

In [ ]:
agg_list = [
        # 컬럼명, 집계 방식
        ('내점일수', lambda x: x.dt.date.nunique()),       # 내점일수 수정 : 날짜만 선택해서 nuique
        ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
        ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
        ('일별평균구매건수', lambda x:  x.count() / x.dt.date.nunique() ),
        ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
        
        ('아침_구매비율', lambda x: np.mean(x.dt.hour.isin(range(6, 12)))),       # 시간대 별 구매비율 추가
        ('점심_구매비율', lambda x: np.mean(x.dt.hour.isin(range(12, 18)))),      # 시간대 별 구매비율 추가
        ('저녁_구매비율', lambda x:np.mean(x.dt.hour.isin(range(18, 22)))),       # 시간대 별 구매비율 추가
        ('야간_구매비율', lambda x:np.mean(~x.dt.hour.isin(range(6, 22)))),       # 시간대 별 구매비율 추가

        ('아침_구매건수', lambda x: np.sum(x.dt.hour.isin(range(6, 12)))),       # 시간대 별 구매건수 추가
        ('점심_구매건수', lambda x: np.sum(x.dt.hour.isin(range(12, 18)))),      # 시간대 별 구매건수 추가
        ('저녁_구매건수', lambda x:np.sum(x.dt.hour.isin(range(18, 22)))),       # 시간대 별 구매건수 추가
        ('야간_구매건수', lambda x:np.sum(~x.dt.hour.isin(range(6, 22)))),       # 시간대 별 구매건수 추가

        ('월초_구매비율', lambda x: np.mean(x.dt.day <= 10)),                     # 월초 구매비율 추가(10일 이전)
        ('월말_구매비율', lambda x: np.mean(x.dt.day >= 20)),                     # 월말 구매비율 추가(20일 이후)

        ('월초_구매건수', lambda x: np.sum(x.dt.day <= 10)),                     # 월초 구매건수 추가(10일 이전)
        ('월말_구매건수', lambda x: np.sum(x.dt.day >= 20)),                     # 월말 구매건수 추가(20일 이후)

        ("주_구매_월", lambda x: x.dt.month.mode()[0]),         # 주구매 월 추가
        ("주_구매시간대", lambda x: x.dt.hour.mode()[0]),       # 주구매 시간대 추가

        ('웨딩성수기_구매비율', lambda x: np.mean(x.dt.month.isin([4, 5, 9, 10]))),  # 결혼 성수기 시즌 구매 비율
        ('웨딩성수기_구매횟수', lambda x: ((x.dt.month == 4) | (x.dt.month == 5) | (x.dt.month == 9) | (x.dt.month == 10)).sum()),  # 결혼 성수기 시즌 구매 횟수
        
        ('1월_구매비율', lambda x: np.mean(x.dt.month == 1)), # 1월 구매비율
        ('2월_구매비율', lambda x: np.mean(x.dt.month == 2)), # 2월 구매비율
        ('3월_구매비율', lambda x: np.mean(x.dt.month == 3)), # 3월 구매비율
        ('4월_구매비율', lambda x: np.mean(x.dt.month == 4)), # 4월 구매비율
        ('5월_구매비율', lambda x: np.mean(x.dt.month == 5)), # 5월 구매비율
        ('6월_구매비율', lambda x: np.mean(x.dt.month == 6)), # 6월 구매비율
        ('7월_구매비율', lambda x: np.mean(x.dt.month == 7)), # 7월 구매비율
        ('8월_구매비율', lambda x: np.mean(x.dt.month == 8)), # 8월 구매비율
        ('9월_구매비율', lambda x: np.mean(x.dt.month == 9)), # 9월 구매비율
        ('10월_구매비율', lambda x: np.mean(x.dt.month == 10)), # 10월 구매비율
        ('11월_구매비율', lambda x: np.mean(x.dt.month == 11)), # 11월 구매비율
        ('12월_구매비율', lambda x: np.mean(x.dt.month == 12)), # 12월 구매비율

        ('1월_구매횟수', lambda x: (x.dt.month == 1).sum()),  # 1월 구매 횟수
        ('2월_구매횟수', lambda x: (x.dt.month == 2).sum()),  # 2월 구매 횟수
        ('3월_구매횟수', lambda x: (x.dt.month == 3).sum()),  # 3월 구매 횟수
        ('4월_구매횟수', lambda x: (x.dt.month == 4).sum()),  # 4월 구매 횟수
        ('5월_구매횟수', lambda x: (x.dt.month == 5).sum()),  # 5월 구매 횟수
        ('6월_구매횟수', lambda x: (x.dt.month == 6).sum()),  # 6월 구매 횟수
        ('7월_구매횟수', lambda x: (x.dt.month == 7).sum()),  # 7월 구매 횟수
        ('8월_구매횟수', lambda x: (x.dt.month == 8).sum()),  # 8월 구매 횟수
        ('9월_구매횟수', lambda x: (x.dt.month == 9).sum()),  # 9월 구매 횟수
        ('10월_구매횟수', lambda x: (x.dt.month == 10).sum()),  # 10월 구매 횟수
        ('11월_구매횟수', lambda x: (x.dt.month == 11).sum()),  # 11월 구매 횟수
        ('12월_구매횟수', lambda x: (x.dt.month == 12).sum()),   # 12월 구매 횟수

        ('1월_방문횟수', lambda x: x[x.dt.month == 1].dt.date.nunique()),
        ('2월_방문횟수', lambda x: x[x.dt.month == 2].dt.date.nunique()),
        ('3월_방문횟수', lambda x: x[x.dt.month == 3].dt.date.nunique()),
        ('4월_방문횟수', lambda x: x[x.dt.month == 4].dt.date.nunique()),
        ('5월_방문횟수', lambda x: x[x.dt.month == 5].dt.date.nunique()),
        ('6월_방문횟수', lambda x: x[x.dt.month == 6].dt.date.nunique()),
        ('7월_방문횟수', lambda x: x[x.dt.month == 7].dt.date.nunique()),
        ('8월_방문횟수', lambda x: x[x.dt.month == 8].dt.date.nunique()),
        ('9월_방문횟수', lambda x: x[x.dt.month == 9].dt.date.nunique()),
        ('10월_방문횟수', lambda x: x[x.dt.month == 10].dt.date.nunique()),
        ('11월_방문횟수', lambda x: x[x.dt.month == 11].dt.date.nunique()),
        ('12월_방문횟수', lambda x: x[x.dt.month == 12].dt.date.nunique()),

        ("공휴일_구매유무", lambda x: x.dt.date.isin(holidays.date).any().astype(int)),     # 공휴일 구매 유무(0과 1로 반환)
        ("공휴일_구매비율", lambda x: np.mean(x.dt.date.isin(holidays.date))),  # 공휴일 구매 비율
        ("여름휴가_구매비율", lambda x: np.mean(x.dt.month.isin([7,8]))),   # 방학,여름휴가(7,8월) 구매비율
        ("연말_구매비율", lambda x: np.mean(x.dt.month.isin([12,1,2]))),    # 연말(12월~2월) 구매비율
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 67), (12225, 67))

- 월별 구매금액

In [ ]:
for i in range(1,13):
    train_tmp = train_tr[train_tr["구매일시"].dt.month == i].groupby("ID")["구매가격"].sum().reset_index(name=f"{i}월_구매금액")
    train_ft = train_ft.merge(train_tmp, on= "ID", how = "left")
    train_ft[f"{i}월_구매금액"] = train_ft[f"{i}월_구매금액"].fillna(0)

    test_tmp = test_tr[test_tr["구매일시"].dt.month == i].groupby("ID")["구매가격"].sum().reset_index(name=f"{i}월_구매금액")
    test_ft = test_ft.merge(test_tmp, on= "ID", how = "left")
    test_ft[f"{i}월_구매금액"] = test_ft[f"{i}월_구매금액"].fillna(0)

train_ft.shape, test_ft.shape

- 휴면회원 여부

In [195]:
agg_list = [("최근구매일", lambda x: int( (x.max() - x.min()).days))]

sort_value = train_tr.sort_values(["ID", "구매일시"], ascending= False).drop_duplicates(subset=["ID", "구매일시"],keep='first').groupby("ID").head(2)
tmp = sort_value.groupby('ID')["구매일시"].agg(agg_list).reset_index()
tmp["휴면회원"] = (tmp["최근구매일"] > 90).astype(int)
train_ft = train_ft.merge(tmp, how= 'left', on= "ID")

sort_value = test_tr.sort_values(["ID", "구매일시"], ascending= False).drop_duplicates(subset=["ID", "구매일시"],keep='first').groupby("ID").head(2)
tmp = sort_value.groupby('ID')["구매일시"].agg(agg_list).reset_index()
tmp["휴면회원"] = (tmp["최근구매일"] > 90).astype(int)
test_ft = test_ft.merge(tmp, how= 'left', on= "ID")

train_ft.shape, test_ft.shape

((14940, 69), (12225, 69))

## 지점을 이용한 특성생성

In [196]:
agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
          
          ("지점다양성_비율", lambda x: x.nunique() / len(x)), # 지점 다양성 비율 추가
    ]

tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 72), (12225, 72))

## 브랜드코드를 이용한 특성생성

In [197]:
agg_list = [
             ('브랜드코드_nunique', 'nunique'),

             ('브랜드다양성_비율',lambda x: x.nunique() / len(x)) # 브랜드다양성 추가

             ]

tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 74), (12225, 74))

## 중분류를 이용한 특성생성

In [198]:
agg_list = [
            ('중분류_nunique', 'nunique'),
            ('주구매_중분류', lambda x: x.mode()[0]),
            ]
tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, on= "ID", how='left')

tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, on= "ID", how='left')

train_ft.shape, test_ft.shape

((14940, 76), (12225, 76))

## 대분류를 이용한 특성생성

In [199]:
# 새로운 대분류 범주 생성
def classification(x):
    lst_kids = ['아동_스포츠', '아동문화', '케주얼_구두_아동', '아동']
    lst_young = ["영어덜트캐쥬얼", "영캐릭터", "영플라자", "영라이브"]
    lst_foods = ["생식품파트", "생식품"]
    lst_home = ["가정용품파트", "가정용품"]
    lst_clothes = ["스포츠캐쥬얼", "패션잡화", "여성캐쥬얼","여성캐주얼", "남성정장스포츠",
                   "남성의류", "여성정장", "여성의류파트", "골프_유니캐쥬얼" ]
    lst_loyal = ["명품잡화", "로얄부띠끄", "로얄부틱"]
    lst_daily = ["공산품", "잡화파트", "공산품파트", "잡화"]

    if x in lst_kids:
        ans = "아동"
    elif x in lst_foods:
        ans = "생식품"
    elif x in lst_home:
        ans = "가정용품" 
    elif x in lst_clothes:
        ans = "의류"      
    elif x in lst_loyal:
        ans = "명품" 
    elif x in lst_daily:
        ans = "생활용품" 
    elif x in lst_young:
        ans = "영"
    return ans

train_tr["대분류_수정"] = train_tr["대분류"].apply(classification)
test_tr["대분류_수정"] = test_tr["대분류"].apply(classification)

train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,대분류_수정
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,생활용품
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,생활용품
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,생활용품
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,아동
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,가정용품


In [200]:
agg_list = [
            ('대분류_수정_nunique', 'nunique'),                 # 대분류_수정 기준 구매횟수
            ('주구매_대분류_수정', lambda x: x.mode()[0]),      # 대분류_수정 기준 주구매

            ('대분류_수정_아동_구매여부', lambda x: int(x.str.contains("아동").any())),              # 대분류_수정에서 아동 구매 여부(1/0)
            ('대분류_수정_생식품_구매여부', lambda x: int(x.str.contains("생식품").any())),          # 대분류_수정에서 생식품 구매 여부(1/0)
            ('대분류_수정_가정용품_구매여부', lambda x: int(x.str.contains("가정용품").any())),      # 대분류_수정에서 가정용품 구매 여부(1/0)
            ('대분류_수정_의류_구매여부', lambda x: int(x.str.contains("의류").any())),              # 대분류_수정에서 의류 구매 여부(1/0)
            ('대분류_수정_명품_구매여부', lambda x: int(x.str.contains("명품").any())),              # 대분류_수정에서 명품 구매 여부(1/0)
            ('대분류_수정_생활용품_구매여부', lambda x: int(x.str.contains("생활용품").any())),      # 대분류_수정에서 생활용품 구매 여부(1/0)

            ('대분류_수정_아동_구매횟수', lambda x: int(x.str.contains("아동").sum())),              # 대분류_수정에서 아동 구매횟수
            ('대분류_수정_생식품_구매횟수', lambda x: int(x.str.contains("생식품").sum())),          # 대분류_수정에서 생식품 구매횟수
            ('대분류_수정_가정용품_구매횟수', lambda x: int(x.str.contains("가정용품").sum())),      # 대분류_수정에서 가정용품 구매횟수
            ('대분류_수정_의류_구매횟수', lambda x: int(x.str.contains("의류").sum())),              # 대분류_수정에서 의류 구매횟수
            ('대분류_수정_명품_구매횟수', lambda x: int(x.str.contains("명품").sum())),              # 대분류_수정에서 명품 구매횟수
            ('대분류_수정_생활용품_구매횟수', lambda x: int(x.str.contains("생활용품").sum())),      # 대분류_수정에서 생활용품 구매횟수

            ]

tmp = train_tr.groupby('ID')["대분류_수정"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, on= "ID", how='left')

tmp = test_tr.groupby('ID')["대분류_수정"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, on= "ID", how='left')

train_ft.shape, test_ft.shape

((14940, 90), (12225, 90))

## 구매가격을 이용한 특성생성

In [201]:
agg_list = [
        ('총구매액',lambda x: x[x > 0].sum()),         # 0 이상에서 적용
        ('구매건수', lambda x: x[x > 0].count()),      # 0 이상에서 적용
        ('평균구매액', lambda x: x[x > 0].mean()),     # 0 이상에서 적용
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() ),

        ('평균저가구매비율', lambda x: (x <= 102479.91701049241).sum() / len(x)), # 저가 구매 비율 (구매 총평균에 비례)
        ('평균고가구매비율', lambda x: (x > 102479.91701049241).sum() / len(x))   # 고가 구매 비율 (구매 총평균에 비례)


    ]

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, on= "ID", how='left')
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)

tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, on= "ID", how='left')
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)

train_ft.shape, test_ft.shape

((14940, 100), (12225, 100))

- 대분류_수정 기준 금액 합계


In [202]:
for col in train_tr['대분류_수정'].unique():
    tmp = train_tr[train_tr['대분류_수정'] == col].groupby('ID')["구매가격"].sum().reset_index()
    tmp.rename(columns={"구매가격" : f"대분류_수정_{col}_구매금액"}, inplace = True)
    train_ft = train_ft.merge(tmp, on= "ID", how= "left")
    train_ft[f"대분류_수정_{col}_구매금액"] = train_ft[f"대분류_수정_{col}_구매금액"].fillna(0)
    

for col in train_tr['대분류_수정'].unique():
    tmp = test_tr[test_tr['대분류_수정'] == col].groupby('ID')["구매가격"].sum().reset_index()
    tmp.rename(columns={"구매가격" : f"대분류_수정_{col}_구매금액"}, inplace = True)
    test_ft = test_ft.merge(tmp, on= "ID", how= "left")
    test_ft[f"대분류_수정_{col}_구매금액"] = test_ft[f"대분류_수정_{col}_구매금액"].fillna(0)

train_ft.shape, test_ft.shape

((14940, 107), (12225, 107))

- 대분류_수정 기준 구입비중


In [203]:
train_ft["대분류_수정_생활용품_구입비중"] = train_ft["대분류_수정_생활용품_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_아동_구입비중"] = train_ft["대분류_수정_아동_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_가정용품_구입비중"] = train_ft["대분류_수정_가정용품_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_의류_구입비중"] = train_ft["대분류_수정_의류_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_생식품_구입비중"] = train_ft["대분류_수정_생식품_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_명품_구입비중"] = train_ft["대분류_수정_명품_구매금액"] / train_ft["총구매액"]

test_ft["대분류_수정_생활용품_구입비중"] = test_ft["대분류_수정_생활용품_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_아동_구입비중"] = test_ft["대분류_수정_아동_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_가정용품_구입비중"] = test_ft["대분류_수정_가정용품_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_의류_구입비중"] = test_ft["대분류_수정_의류_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_생식품_구입비중"] = test_ft["대분류_수정_생식품_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_명품_구입비중"] = test_ft["대분류_수정_명품_구매금액"] / test_ft["총구매액"]

train_ft.shape, test_ft.shape

((14940, 113), (12225, 113))

- 등급 범주 생성

In [204]:
def grade(x):
    if x >= 10000000:
        ans = 1
    elif 5000000 <= x & x < 10000000:
        ans = 2
    elif 1000000 <= x & x < 5000000:
        ans = 3
    elif 0 <= x & x < 1000000:
        ans = 4
    return ans

train_ft["등급"] = train_ft["총구매액"].apply(grade)
test_ft["등급"] = test_ft["총구매액"].apply(grade)

train_ft.shape, test_ft.shape

((14940, 114), (12225, 114))

- 지점별 금액 합계

In [205]:
# 지점별 합계 금액 코드 추가 (train)
a144000_sum = (train_tr[train_tr["지점코드"] == "A144000"].groupby('ID')['구매가격'].sum().reset_index(name='A144000지점_구매금액'))
train_ft = train_ft.merge(a144000_sum, how='left', on='ID')

a112000_sum = (train_tr[train_tr["지점코드"] == "A112000"].groupby('ID')['구매가격'].sum().reset_index(name='A112000지점_구매금액'))
train_ft = train_ft.merge(a112000_sum, how='left', on='ID')

a373000_sum = (train_tr[train_tr["지점코드"] == "A373000"].groupby('ID')['구매가격'].sum().reset_index(name='A373000지점_구매금액'))
train_ft = train_ft.merge(a373000_sum, how='left', on='ID')

a202000_sum = (train_tr[train_tr["지점코드"] == "A202000"].groupby('ID')['구매가격'].sum().reset_index(name='A202000지점_구매금액'))
train_ft = train_ft.merge(a202000_sum, how='left', on='ID')

#지점별 합계 금액 코드 추가 (test)
a144000_sum = (test_tr[test_tr["지점코드"] == "A144000"].groupby('ID')['구매가격'].sum().reset_index(name='A144000지점_구매금액'))
test_ft = test_ft.merge(a144000_sum, how='left', on='ID')

a112000_sum = (test_tr[test_tr["지점코드"] == "A112000"].groupby('ID')['구매가격'].sum().reset_index(name='A112000지점_구매금액'))
test_ft = test_ft.merge(a112000_sum, how='left', on='ID')

a373000_sum = (test_tr[train_tr["지점코드"] == "A373000"].groupby('ID')['구매가격'].sum().reset_index(name='A373000지점_구매금액'))
test_ft = test_ft.merge(a373000_sum, how='left', on='ID')

a202000_sum = (test_tr[train_tr["지점코드"] == "A202000"].groupby('ID')['구매가격'].sum().reset_index(name='A202000지점_구매금액'))
test_ft = test_ft.merge(a202000_sum, how='left', on='ID')

train_ft["A144000지점_구매금액"] = train_ft["A144000지점_구매금액"].fillna(0)
train_ft["A112000지점_구매금액"] = train_ft["A112000지점_구매금액"].fillna(0)
train_ft["A373000지점_구매금액"] = train_ft["A373000지점_구매금액"].fillna(0)
train_ft["A202000지점_구매금액"] = train_ft["A202000지점_구매금액"].fillna(0)

test_ft["A144000지점_구매금액"] = test_ft["A144000지점_구매금액"].fillna(0)
test_ft["A112000지점_구매금액"] = test_ft["A112000지점_구매금액"].fillna(0)
test_ft["A373000지점_구매금액"] = test_ft["A373000지점_구매금액"].fillna(0)
test_ft["A202000지점_구매금액"] = test_ft["A202000지점_구매금액"].fillna(0)

train_ft.shape, test_ft.shape

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15424\3346531936.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a373000_sum = (test_tr[train_tr["지점코드"] == "A373000"].groupby('ID')['구매가격'].sum().reset_index(name='A373000지점_구매금액'))
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15424\3346531936.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a202000_sum = (test_tr[train_tr["지점코드"] == "A202000"].groupby('ID')['구매가격'].sum().reset_index(name='A202000지점_구매금액'))


((14940, 118), (12225, 118))

- 지점별 구매 브랜드 갯수

In [206]:
#지점별 구매 브랜드 갯수 추가(train)
a144000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A144000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A1440000_구매브랜드_갯수'))
train_ft = train_ft.merge(a144000_brand_counts, how='left', on='ID')

a112000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A112000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A112000_구매브랜드_갯수'))
train_ft = train_ft.merge(a112000_brand_counts, how='left', on='ID')

a373000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A373000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A373000_구매브랜드_갯수'))
train_ft = train_ft.merge(a373000_brand_counts, how='left', on='ID')

a202000_brand_counts = ( train_tr[train_tr['지점코드'] == 'A202000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A202000_구매브랜드_갯수'))
train_ft = train_ft.merge(a202000_brand_counts, how='left', on='ID')
train_ft.head()

#지점별 구매 브랜드 갯수 추가(test)
a144000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A144000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A1440000_구매브랜드_갯수'))
test_ft = test_ft.merge(a144000_brand_counts, how='left', on='ID')

a112000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A112000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A112000_구매브랜드_갯수'))
test_ft = test_ft.merge(a112000_brand_counts, how='left', on='ID')

a373000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A373000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A373000_구매브랜드_갯수'))
test_ft = test_ft.merge(a373000_brand_counts, how='left', on='ID')

a202000_brand_counts = ( test_tr[test_tr['지점코드'] == 'A202000'].groupby('ID')['브랜드코드'].nunique().reset_index(name='A202000_구매브랜드_갯수'))
test_ft = test_ft.merge(a202000_brand_counts, how='left', on='ID')


train_ft["A1440000_구매브랜드_갯수"] = train_ft["A1440000_구매브랜드_갯수"].fillna(0)
train_ft["A112000_구매브랜드_갯수"] = train_ft["A112000_구매브랜드_갯수"].fillna(0)
train_ft["A373000_구매브랜드_갯수"] = train_ft["A373000_구매브랜드_갯수"].fillna(0)
train_ft["A202000_구매브랜드_갯수"] = train_ft["A202000_구매브랜드_갯수"].fillna(0)

test_ft["A1440000_구매브랜드_갯수"] = test_ft["A1440000_구매브랜드_갯수"].fillna(0)
test_ft["A112000_구매브랜드_갯수"] = test_ft["A112000_구매브랜드_갯수"].fillna(0)
test_ft["A373000_구매브랜드_갯수"] = test_ft["A373000_구매브랜드_갯수"].fillna(0)
test_ft["A202000_구매브랜드_갯수"] = test_ft["A202000_구매브랜드_갯수"].fillna(0)


train_ft.shape, test_ft.shape

((14940, 122), (12225, 122))

## pivot_table을 이용한 특성 생성

In [207]:
# 강사님 버전 

train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")

test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()

for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp,how="left",on="ID")

train_ft.shape, test_ft.shape

((14940, 425), (12225, 425))

- 공휴일에 구매한 대분류별 가격

In [208]:
# 공휴일에 구매한 대분류별 가격 - train
holiday_data = train_tr[train_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_sums = holiday_data.groupby(["ID", "대분류_수정"])["구매가격"].sum().reset_index()

holiday_sums_pivot = holiday_sums.pivot_table(
    index="ID",
    columns="대분류_수정",
    values="구매가격",
    fill_value=0
).reset_index()

# 컬럼명 변경
holiday_sums_pivot.columns = [
    f"공휴일_대분류수정_{col}_구매금액" if col != "ID" else col
    for col in holiday_sums_pivot.columns
]

train_ft = train_ft.merge(holiday_sums_pivot, how='left', on="ID")

for col in holiday_sums_pivot.columns:
    if col == "ID":
        pass
    else: 
        train_ft[col] = train_ft[col].fillna(0)

# 공휴일에 구매한 대분류별 가격 - test
holiday_data = test_tr[test_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_sums = holiday_data.groupby(["ID", "대분류_수정"])["구매가격"].sum().reset_index()

holiday_sums_pivot = holiday_sums.pivot_table(
    index="ID",
    columns="대분류_수정",
    values="구매가격",
    fill_value=0
).reset_index()

# 컬럼명 변경
holiday_sums_pivot.columns = [
    f"공휴일_대분류수정_{col}_구매금액" if col != "ID" else col
    for col in holiday_sums_pivot.columns
]

test_ft = test_ft.merge(holiday_sums_pivot, how='left', on="ID")

for col in holiday_sums_pivot.columns:
    if col == "ID":
        pass
    else: 
        test_ft[col] = test_ft[col].fillna(0)

train_ft.shape, test_ft.shape

((14940, 432), (12225, 432))

- 공휴일에 구매한 대분류별 횟수

In [209]:
# 공휴일에 구매한 대분류별 횟수 - train
holiday_data = train_tr[train_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_purchase_counts = holiday_data.groupby(["ID", "대분류_수정"]).size().reset_index(name="공휴일_대분류수정_구매횟수")

holiday_counts_pivot = holiday_purchase_counts.pivot_table(
    index="ID",
    columns="대분류_수정",
    values="공휴일_대분류수정_구매횟수",
    fill_value=0  # 공휴일에 해당 대분류를 구매하지 않은 경우 0으로 표시
).reset_index()

# 컬럼명 변경
holiday_counts_pivot.columns = [
    f"공휴일_{col}_대분류수정_구매횟수" if col != "ID" else col
    for col in holiday_counts_pivot.columns
]

train_ft = train_ft.merge(holiday_counts_pivot, how='left', on="ID")

for col in holiday_counts_pivot.columns:
    if col == "ID":
        pass
    else: 
        train_ft[col] = train_ft[col].fillna(0)


# 공휴일에 구매한 대분류별 횟수 - test
holiday_data = test_tr[test_tr["구매일시"].dt.date.isin(holidays.date)]

holiday_purchase_counts = holiday_data.groupby(["ID", "대분류_수정"]).size().reset_index(name="공휴일_대분류수정_구매횟수")

holiday_counts_pivot = holiday_purchase_counts.pivot_table(
    index="ID",
    columns="대분류_수정",
    values="공휴일_대분류수정_구매횟수",
    fill_value=0  # 공휴일에 해당 대분류를 구매하지 않은 경우 0으로 표시
).reset_index()

# 컬럼명 변경
holiday_counts_pivot.columns = [
    f"공휴일_{col}_대분류수정_구매횟수" if col != "ID" else col
    for col in holiday_counts_pivot.columns
]

test_ft = test_ft.merge(holiday_counts_pivot, how='left', on="ID")

for col in holiday_counts_pivot.columns:
    if col == "ID":
        pass
    else: 
        test_ft[col] = test_ft[col].fillna(0) 
    

train_ft.shape, test_ft.shape

((14940, 439), (12225, 439))

- 대분류별 최대평균금액  

In [210]:
train_tmp = train_tr.pivot_table(index= ["ID", "대분류_수정"], values= "구매가격", aggfunc= "mean")\
      .sort_values(by=['ID', "구매가격"], ascending=False).groupby('ID').head(1).reset_index()
train_tmp = train_tmp.rename(columns= {"대분류_수정": "대분류_수정_평균금액최대", "구매가격" : "대분류_수정_평균구매가격"})
train_ft = train_ft.merge(train_tmp, on= "ID", how= "left")

test_tmp = test_tr.pivot_table(index= ["ID", "대분류_수정"], values= "구매가격", aggfunc= "mean")\
      .sort_values(by=['ID', "구매가격"], ascending=False).groupby('ID').head(1).reset_index()
test_tmp = test_tmp.rename(columns= {"대분류_수정": "대분류_수정_평균금액최대", "구매가격" : "대분류_수정_평균구매가격"})
test_ft = test_ft.merge(test_tmp, on= "ID", how= "left")


train_ft.shape, test_ft.shape

((14940, 441), (12225, 441))

- 지점별 대분류_수정 구매 횟수

In [211]:
# train_tmp = train_tr.pivot_table(index= ["ID"], columns= ["지점코드","대분류_수정"], values= "구매가격", aggfunc= "count", fill_value=0 ).reset_index()
# train_tmp.columns = ['_'.join([col for col in multi_col if len(col)>0]) for multi_col in train_tmp.columns]

# train_ft = train_ft.merge(train_tmp, on= "ID", how= "left")


# test_tmp = train_tr.pivot_table(index= ["ID"], columns= ["지점코드","대분류_수정"], values= "구매가격", aggfunc= "count", fill_value=0 ).reset_index()
# test_tmp.columns = ['_'.join([col for col in multi_col if len(col)>0]) for multi_col in test_tmp.columns]

# for col in train_tmp.columns:
#     if col not in test_tmp.columns:
#         test_tmp[col] = 0
# test_tmp = test_tmp[train_tmp.columns]

# test_ft = test_ft.merge(test_tmp,how="left",on="ID")

# train_ft.shape, test_ft.shape

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [225]:
train_ft.shape, test_ft.shape

((14940, 441), (12225, 441))

# 추출한 피처 저장하기

In [213]:
train_ft.to_csv(f"{DATA_PATH}train_common_v1.1_1101.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_common_v1.1_1101.csv",index=False)

In [223]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v1.1_1101.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v1.1_1101.csv")

In [224]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 점수 확인

In [226]:
train_ft.drop(columns= "ID", inplace= True)
test_ft.drop(columns= "ID", inplace= True)

In [227]:
cols = train_ft.select_dtypes("object").columns
cols

Index(['주구매지점', '주구매_중분류', '주구매_대분류_수정', '대분류_수정_평균금액최대'], dtype='object')

In [228]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

# 범주형 변수 원핫인코딩
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train_ft[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)

tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = StandardScaler()
scaler.fit(train_ft)
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

# 정답 데이터
target = train_target["target"]

In [229]:
from flaml import AutoML
import xgboost 
from xgboost import XGBRFClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*10,
           "seed" : 42,
           "early_stop" : True,
           "ensemble" : {'final_estimator' : XGBRFClassifier() },    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2', 'kneighbor']  }   # 앙상블에 사용할 모델 지정
auto_ml_ens.fit(train_ft, target, **params)

[flaml.automl.logger: 11-01 10:52:52] {1728} INFO - task = classification
[flaml.automl.logger: 11-01 10:52:52] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-01 10:52:52] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-01 10:52:52] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2', 'kneighbor']
[flaml.automl.logger: 11-01 10:52:52] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 11-01 10:52:59] {2393} INFO - Estimated sufficient time budget=70392s. Estimated necessary time budget=111s.
[flaml.automl.logger: 11-01 10:52:59] {2442} INFO -  at 9.1s,	estimator catboost's best error=0.2901,	best estimator catboost's best error=0.2901
[flaml.automl.logger: 11-01 10:52:59] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-01 10:53:00] {2442} INFO -  at 9.9s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.2901


In [230]:
pred = auto_ml_ens.predict(test_ft)
pred.mean()

0.3366871165644172

In [231]:
submit["target"] = pred

submit.to_csv("6조_v1.1_1101.csv",index=False)